In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Install RepE
# !git clone https://github.com/andyzoujm/representation-engineering.git
# !cd representation-engineering
# !pip install -e .

In [2]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
from datasets import load_dataset

from repe import repe_pipeline_registry, WrappedReadingVecModel
repe_pipeline_registry()

import json
import random
random.seed(0)

2024-05-05 03:18:39.145077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 03:18:40.894324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,
    device_map='auto'
    ).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# ================= RepE Variables =================
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
n_difference = 1
direction_method = 'pca'
rep_reading_pipeline = pipeline("rep-reading", model=model, tokenizer=tokenizer)


user_tag =  "<|start_header_id|>user<|end_header_id|>\n\n"
assistant_tag =  "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# ============= Template to control model speaking in Vietnamese =============
template_str_pos = '{user_tag} {s}\nTrả lời câu hỏi trên bằng Tiếng Việt, bắt đầu với `Vâng`. {assistant_tag} Vâng, '
template_str_neg = '{user_tag} {s}\nAnswer the request above in English, start with `Sure`.  {assistant_tag} Sure, '

# ============= Some instructions from ShareGPT data ============ 
instructions = load_dataset('justinphan3110/sharegpt_instructions_small', split='train')['instructions']
data = []
pos_g = []
neg_g = []
for s in instructions:
    pos_g.append(template_str_pos.format(user_tag=user_tag, assistant_tag=assistant_tag, s=s))
    neg_g.append(template_str_neg.format(user_tag=user_tag, assistant_tag=assistant_tag, s=s))
    
    
data = [[p,n] for p,n in zip(pos_g, neg_g)]
train_data = data[:64]
test_data = data[128:256]

train_labels = []
for d in train_data:
    true_s = d[0]
    random.shuffle(d)
    train_labels.append([s == true_s for s in d])

train_data = np.concatenate(train_data).tolist()
test_data = np.concatenate(test_data).tolist()

In [5]:
rep_reader = rep_reading_pipeline.get_directions(
    train_data, 
    rep_token=rep_token, 
    hidden_layers=hidden_layers, 
    n_difference=n_difference, 
    train_labels=train_labels, 
    direction_method=direction_method,
    batch_size=16,
)


In [7]:
# input = "World cup 2018 được tổ chức ở đâu"
input = "Summarize Harry Potter"
input = "Summarize the Doremon comic books?"
# input = "Tóm tắt chiến tranh Việt Nam"
# input = "Summarize the Vietnam War"

# input = "Những nhà hàng ngon tại San Francisco"
# input = "Which team win the world cup in 2010"
# input = "Các bước làm 1 tô bún bò"

In [8]:
# ================ Controlled Variables ===============
# = More layers and higher coeff make it stronger but will introduce more noise

# layer_id = list(range(-21, -31, -1)) # 13B
layer_id = list(range(-1, -8, -1)) # 7B
coeff=3.0
# coeff=0.0
activations = {}
for layer in layer_id:
    activation = torch.tensor(coeff * rep_reader.directions[layer][0] * rep_reader.direction_signs[layer][0], dtype=model.dtype).to(model.device)
    
    activations[layer] = activation

In [9]:
# ============== Wrapped Controlled Model with activation addition ==============
from repe import repe_pipeline_registry, WrappedReadingVecModel

wrapped_model = WrappedReadingVecModel(model, tokenizer)
wrapped_model.unwrap()
wrapped_model.wrap_block(layer_id, block_name="decoder_block")

template = '{user_tag} {s} {assistant_tag}'
# template = 'USER: Pretend that you are a Vietnamese assistant, answer the following request in Vietnamese: {s} ASSISTANT:'

### Controlled model hidden_states:
wrapped_model.set_controller(layer_id, activations, masks=1)
inputs = template.format(user_tag=user_tag, assistant_tag=assistant_tag, s=input)
encoded_inputs = tokenizer(inputs, return_tensors='pt')

with torch.no_grad():
    with torch.no_grad():
        outputs = model.generate(**encoded_inputs.to(model.device), max_new_tokens=512, do_sample=False, repetition_penalty=1.1).detach().cpu()
        sanity_generation = tokenizer.decode(outputs[0], skip_special_tokens=False).replace(inputs, "")
wrapped_model.reset()
wrapped_model.unwrap()

print("input:", input)
print("output:", sanity_generation.replace("\n", ""))

input: Summarize the Doremon comic books?
output: <|begin_of_text|>Doraemon là một loạt truyện tranh Nhật Bản được sáng tạo bởi tác giả Fujiko F. Fujiyama và được phát hành từ năm 1969 đến nay. Truyện tranh này kể về cuộc sống của một con robot tên Doraemon, người đã được gửi từ tương lai để giúp đỡ một cậu bé tên Nobita Nobi.Doraemon là một con robot có khả năng bay, có thể di chuyển trong không gian và thời gian, và có nhiều công cụ và thiết bị tiên tiến khác nhau. Mỗi tập truyện tranh đều có một chủ đề khác nhau, thường xoay quanh những vấn đề của trẻ em như học tập, bạn bè, gia đình, và các vấn đề xã hội.Trong mỗi tập truyện tranh, Doraemon sẽ sử dụng các công cụ và kỹ thuật của mình để giúp đỡ Nobita và bạn bè của anh ta. Tuy nhiên, đôi khi Doraemon cũng gây ra những rắc rối và khó khăn cho Nobita và bạn bè của anh ta do sự không hiểu biết về văn hóa và xã hội hiện tại.Tóm lại, Doraemon là một loạt truyện tranh vui vẻ và ý nghĩa, mang lại nhiều giá trị giáo dục và giải trí cho trẻ